In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
import math
from sklearn.metrics.pairwise import rbf_kernel
from numpy import genfromtxt

In [2]:
X = pd.read_csv("life_expectancy_X.csv").to_numpy()
Y = pd.read_csv("life_expectancy_y.csv").to_numpy()
n = len(Y)
Y = np.reshape(Y,(n,1))

In [3]:
# Adding Bias to X
X = np.hstack((X, np.zeros((X.shape[0], 1))))
# Scale the data between 0 and 1
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

Use HSIC between each feature and label to identify the features that have the strongest dependency on the label.

In [21]:
def HSIC(X, Y):
    γ = 1 
    K_x = rbf_kernel(X, gamma=1)
    K_y = rbf_kernel(X, gamma=1)
    C = np.eye(n) - (1/n) * np.ones((n,n))
    return np.trace(K_x.dot(C).dot(K_y).dot(C))/ (n*n)

features = ['Exercise amount', 
            'Amount of supportive relationships' ,
            'Number of siblings', 
            'Alcohol / Drugs / Smoking consumption', 
            'Height', 'Attractiveness' ,'work ethicsm', 'bias']

HSIC_list = []
for f in X.T:
    f = np.reshape(f, (n,1))
    HSIC_list.append(HSIC(f, Y))


In [23]:
df = pd.DataFrame([HSIC_list], columns=features)
# Display the DataFrame
print(df)

   Exercise amount  Amount of supportive relationships  Number of siblings  \
0         0.015822                            0.017959            0.026307   

   Alcohol / Drugs / Smoking consumption    Height  Attractiveness  \
0                               0.016401  0.016604        0.016032   

   work ethicsm          bias  
0      0.015122  7.476939e-31  


Use ISM to identify the most important features.